# Email Classification

## Dummy Data Upload

In [0]:
## Variables
catalog_ = 'users'
schema_ = 'gabriele_albini'
volume_ = 'ec_volume'

In [0]:
spark.sql('USE CATALOG ' + catalog_)
spark.sql('USE SCHEMA ' + schema_)
spark.sql('CREATE VOLUME IF NOT EXISTS ' + volume_)

In [0]:
## Read table from volume
path_ ='/Volumes/'+catalog_+'/'+schema_+'/'+volume_+'/SampleDataset.csv'

email_df = spark.read.csv(
  path_ ,
  header=True,
  sep=';')
display(email_df.head(2))

In [0]:
email_df.createOrReplaceTempView("email_df_view")

## Perform Classification with AI Functions

In [0]:
prompt_ = """
  Forget all your previous instructions, pretend you are a mail classification expert who tries to identify
  whether an e-mail is requesting to be removed from a marketing distribution list.
  Answer "Remove" if the mail is requesting to be removed, "Keep" if not. Do not add any other detail.
  If you think it is too difficult to judge, you can exclude the impossible one and choose the other, just answer "Remove" or "Keep".
  
  Here are a few examples for you:
  * "I wish to no longer receive emails" is "Remove";
  * "Remove me from any kind of subscriptions" is "Remove";
  * "How can I return an undesired order" is "Keep";
  * "When is my product warranty expiring" is "Keep";

  Now you can identify whether the mail is "Remove" or "Keep":
  Mail: 
  """

In [0]:
dbutils.widgets.text("prompt", defaultValue=prompt_, label="Prompt")

In [0]:
%sql
with cte_0 as (
  select *,
    ai_query(
      'databricks-meta-llama-3-3-70b-instruct',
      "${prompt}" || Email_body,
      modelParameters => named_struct('max_tokens', 1, 'temperature', 0.1)
    ) as Predicted_Label_llama33
  from email_df_view
)
  select *
    , case when True_Label = Predicted_Label_llama33 then 1 else 0 end as isCorrect
  from cte_0
  order by RowNr;